In [ ]:
%matplotlib widget
#%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")
#set_matplotlib_formats('svg')
# plt.rc('legend', frameon=False)
# plt.rc('figure', figsize=(7, 7 / 1.75)) # Larger figure sizes
# plt.rc('font', size=12)

# With the Bifrost Snapshot or with existing atmosperes' models

```
# There are four 1D models of flares on M-dwarfs (/mn/stornext/u3/matsc/rh/Atmos/mflares/). 
# These are:

# radyn_out.F13_dpl_GRID_pt1s.ncdf
# tx.m2F12-37.max2F12_Ec37_d3.ncdf
# tx.m5F12-37.max5F12_Ec37_d3.ncdf
# tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf

# the three "tx" models all have a low cut-off energy of 37 keV and a power index of 3 
# (see Allred et al. 2015) 
# but vary in maximum beam energy: 2e12, 5e12 and 1e13 erg/cm^2/s. 

#The model from Kowalski et al. 2015 is radyn_out.F13_dpl_GRID_pt1s.ncdf.
```



In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/code/rh/Atmos/"

In [ ]:
# to read xarray.Dataset and see dimentions, coordinates, data variables and attributes 
import xarray
atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos
# out = xarray.open_dataset("/Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5")
# out
#output_indata.hdf5 file.

#output_ray.hdf5 file.

```
#netcdf4 to hdf5
```

In [ ]:
atmos.to_netcdf("newfile.hdf5", format='NETCDF4')

```
# виджет хелиты все равно не работает даже с переделанным файлом из netcdf
# строит только 1 график
```

In [ ]:
from helita.vis import rh15d_vis

# hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# H = hf['z'][0,:]
#print(H)
rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/repository/jupyter/newfile.hdf5")



# Plotting atmospheres in nz

In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/code/rh/Atmos"

In [ ]:
#hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf,"r")
height = hf['z'][0,:,0,:] # geometric height
time = hf['x'][:]
#print(height.shape())
temp = hf['temperature']
vlos = hf['velocity_z'] # LOS_V
# eden = hf['electron_density']

# snapshot_number = hf["snapshot_number"]
# nhydr = hf["nhydr"]
# x = hf["x"]
# z = hf["z"]
print(height)


In [ ]:
# to plot 2-d atmospheres with height scale as index 
fig, ax = plt.subplots()
#atmos.temperature.plot.hist()
#atmos.temperature.plot()
atmos.velocity_z.plot()
#atmos.electron_density.plot()
#im=axs[0].imshow(temp[0,:,0,:])

#ax.plot(height / 1e6, time)  # height in Mm
ax.set_xlabel('H (Mm)')
ax.set_ylabel('time')
#ax.set_yscale('log')
#ax.set_xscale('log')

#ax.set_xlabel('height')
#ax.set_ylabel('time')


```
# additional way to see variables
```

In [ ]:
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

In [ ]:
# To see what data is in this file, we can call the keys() method on the file object.
hf.keys() # variables in the Bifrost snapshot

# Plotting atmospheres in real height

In [ ]:
#from helita.sim import rh15d_vis
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
H = hf['z'][0,:]
#print(H)
rh15d_vis.InputAtmosphere2D("/Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5")
#nz = H
#atmos.temperature.plot.hist()
#fig, ax = plt.subplots()
#atmos.temperature.plot()
#atmos.velocity_z.plot()

# atmos.electron_density.plot()
# ax.set_xlabel('height')
# ax.set_ylabel('time')
#hf = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
#hf.keys()
# atmos = hf['atmos']
# height = hf['height']
# inputfile = hf['input']
# mpi = hf['mpi']

# print(atmos)
# print("///")
# print(height)
# print("///")
# print(inputfile)
# print("///")
# print(mpi)

#Blos = hf['B_z'] # LOS B
#H = hf['z'][0,:] # geometric height
#snapshot_number = hf["snapshot_number"]
#nhydr = hf["nhydr"]

#import xarray
#inputs = xarray.open_dataset(rhRepoPath / "rh15d/run/output/output_indata.hdf5")


In [ ]:
%matplotlib widget


hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, ax=plt.subplots()

# ax1.plot(z1t[:,0],np.log10(tg1t[:,100]),label='10.0 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,75]),label='7.5 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,60]),label='6.0 sec')

# ax.plot(z1t[:,0],np.log10(tg1t[:,40]),'k--', label='4.0 sec')
# ax.plot(z1t[:,0],np.log10(tg1t[:,22]),'k-',label='2.2 sec')
# ax.plot(z1t[:,0],np.log10(tg1t[:,12]),'r-.',label='1.2 sec')
# ax.plot(z1t[:,0],np.log10(tg1t[:,2]),'b-',label='0.2 sec')
# ax.plot(z1t[:,0],np.log10(tg1t[:,1]),'k-',label='0.1 sec')

# ax.plot(z1t[:,0],np.log10(tg1t[:,0]),'g-',label='0 sec')

# ax.plot(z1t[:,0],vz1t[:,135],label='13.5 sec')
# ax.plot(z1t[:,0],vz1t[:,90],label='9.0 sec')
# ax.plot(z1t[:,0],vz1t[:,60],label='6.0 sec')

# ax.plot(z1t[:,0],vz1t[:,40],label='4.0 sec')
# ax.plot(z1t[:,0],vz1t[:,22],label='2.2 sec')
# ax.plot(z1t[:,0],vz1t[:,12],label='0.2 sec')
ax.plot(z1t[:,0],vz1t[:,2],label='0.2 sec')

ax.plot(z1t[:,0],vz1t[:,1],label='0.1 sec')
ax.plot(z1t[:,0],vz1t[:,0],label='0 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,130]),label='13.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,100]),label='10.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,75]),label='7.5 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,40]),label='4.0 sec')
#ax.plot(z1t[:,0],np.log10(ed1t[:,22]),label='2.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,12]),label='1.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,2]),label='0.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,0]),label='0 sec')




plt.xlabel('Height (Mm)')

# plt.ylabel('lg(Temperature (K))')
plt.ylabel('Velocity (m/s)')
# plt.xlim(-0.2,0.5)
# plt.ylim(-1e5,2e5)
#plt.ylabel('lg(Electron Density ())')
#plt.title('m2F12-37 Atmos’)
ax.legend()  # Add a legend.


# plt.savefig("temperature_.svg", format="svg")
plt.savefig("velocity_z_.svg", format="svg")
# plt.savefig("electron_density_.svg", format="svg")


# Atoms for CaII

```
# Nmetal
   10

# Metals
#  model file              ACTIVE/PASSIVE  INITIAL_SOLUTION   population file


# ../../Atoms/CaII.atom            PASSIVE    ZERO_RADIATION
  ../../Atoms/H_6.atom             PASSIVE    ZERO_RADIATION
  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION
  ../../Atoms/CaII_PRD.atom        ACTIVE     ZERO_RADIATION
  ../../Atoms/Si.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Al.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Fe.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/He.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/N.atom               PASSIVE    LTE_POPULATIONS
  ../../Atoms/Na.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/S.atom               PASSIVE    LTE_POPULATIONS
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

#wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

# Create own wavetable file

```
# Writes RH wave file (in xdr format). All wavelengths should be in nm.

# Parameters
# ----------
# start: number
#     Starting wavelength.
# end: number
#     Ending wavelength (non-inclusive)
# step: number
#     Wavelength separation
# outfile: string
#     Name of file to write.
# ewave: 1-D array, optional
#    Array of existing wavelengths. Program will make discard points
#    to make sure no step is enforced using these points too.
# air: boolean, optional
#     If true, will at the end convert the wavelengths into vacuum
#     wavelengths.
```

In [ ]:
def make_wave_file(my_wave, start=20, end=4860, step=4, new_wave=None,
                   ewave=None, air=False):
                   rh15d.make_wave_file('my_wave.wave', ewave=my_waves)

In [ ]:
rh15d.make_wave_file('My_wave.wave', 20, 4290, 3.5)

In [ ]:
#from helita.sim import rh15d
#read exisitng file if needed
MY_wave = rh15d.read_wave_file(rhRepoPath / "Atoms/wave_files/My_wave.wave")


In [ ]:
#import xarray
#atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
print(MY_wave)

In [ ]:
MY_wave.shape

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 200) & (wave < 500)]
#If we want to see also one particular wavelength (500 in example)
wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

# Intencity plot for Ca II

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

In [ ]:
%matplotlib widget
#%matplotlib inline
data.ray.intensity.plot()
#plt.legend(loc="upper left")
plt.xlabel("Wavelenght nm")
plt.ylabel("Time, s*10")
#plt.axis([20, 1000, 0, 62])

```
# to see what's in the output data
```

In [ ]:
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()
# tau1.plot()

```
# in order for the graphs to work properly, we make indices for wavelenghts array
```

In [ ]:
%matplotlib widget
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 20) & (wave < 4000)]
#If we want to see also one particular wavelength (500 in example)
# wave.sel(wavelength=500, method="nearest")
# index500 = np.argmin(np.abs(wave.data - 500))



# Tau=1 height

In [ ]:
%matplotlib widget

fig, ax=plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6, 'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6, 'r-',label='0.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6, 'r-.',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6, 'b--',label='2.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6, 'b-.',label='4. sec')

ax.legend(loc="upper right") 
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')
plt.title(r'$\tau=1$')
plt.savefig("Tau_one_height_.svg", format="svg")


```
#To make a plot for intensity at (x, y) = (?, 0), x - time column
```

In [ ]:
%matplotlib widget

fig, ax=plt.subplots()
#plt.figure()
# for i in range(0,2,12,22,38):
#     plt.plot(
#         data.ray.intensity[i, 0],
#         label='Time %i' % (i)
#     )
# data.ray.intensity[100, 0].plot(label='10 sec')
# data.ray.intensity[75, 0].plot(label='7.5 sec')
# data.ray.intensity[60, 0].plot(label='6 sec')

ax.plot(wave[indices],data.ray.intensity[0,0,indices],'k--',label='0 sec')
ax.plot(wave[indices],data.ray.intensity[2,0,indices],'k-',label='0.2 sec')
ax.plot(wave[indices],data.ray.intensity[12,0,indices],'r--',label='1.2 sec')
ax.plot(wave[indices],data.ray.intensity[22,0,indices],'r-',label='2.2 sec')
ax.plot(wave[indices],data.ray.intensity[40,0,indices],'k-.',label='4 sec')


# data.ray.intensity[40, 0].plot(label='4 sec')
# data.ray.intensity[22, 0].plot(label='2.2 sec')
# data.ray.intensity[12, 0].plot(label='1.2 sec')
# data.ray.intensity[2, 0].plot(label='0.2 sec')
# data.ray.intensity[0, 0].plot(label='0 sec')


ax.legend()  # Add a legend.

plt.savefig("intensity_.svg", format="svg")
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
plt.axis([20, 4000, 0, 8e-7])
plt.savefig("Intencity_.svg", format="svg")



```
#Flux from Intencity
```

In [ ]:
import scipy.constants

print("c = %s"%str(scipy.constants.physical_constants["speed of light in vacuum"]))

```
# Calculating Flux from Intensity in dimensions W/m^2/Hz in 2D array, file:fluxes2d.npy
# Calculating Surface Flux in erg/cm^2/s/å in 2D array, file:luxesSurface2d.npy
```

In [ ]:
data.ray.intensity.wavelength[0].data*10

In [ ]:
#import scipy.constants
from scipy.integrate import quad, dblquad

fluxes = []
fluxesSurface = []
# const for dimension converting
const1 = 2.99792458e+21
#const1 = scipy.constants.physical_constants["speed of light in vacuum"]*
# uncomment for all columns calculation - takes quite some time
#for i in range(data.ray.intensity.x.count().data):
for i in [0,2,12,22,40]:
# for i in [0]:
    for ii in range(data.ray.intensity.wavelength.count().data):
        flx, err = quad(lambda x: np.sin(x)*np.cos(x)*data.ray.intensity[i,0,ii].data, 0, np.pi/2)

# for i in range(data.ray.intensity.x.count().data):
# for i in [0,2,12,22,40]:
# for i in [12]:
#     for ii in range(data.ray.intensity.wavelength.count().data):
#         flx, err = dblquad(lambda x, y: np.sin(x)*np.cos(x)*data.ray.intensity[i,0,ii].data, 0, 2*np.pi, lambda x: 0, lambda x: np.pi/2)
        fluxes.append(flx * 2.0 * np.pi)
        # fluxes.append(flx)
        fluxesSurface.append(const1 * flx /(np.power(data.ray.intensity.wavelength[ii].data * 10,2))


In [ ]:
# Write fluxes from 1D to 2D arrays
fluxesNP = np.array(fluxes)
fluxes2d = fluxesNP.reshape(5,1220)
print(fluxes2d.shape, fluxes2d[0,0])
c

fluxesSurfaceNP = np.array(fluxesSurface)
fluxesSurface2d = fluxesSurfaceNP.reshape(5,1220)
print(fluxesSurface2d.shape, fluxesSurface2d[0,0])
np.save("fluxesSurface2d.npy", fluxesSurface2d)

In [ ]:
print(fluxes2d.shape, fluxes2d[0,50])
print(fluxesSurface2d.shape, fluxesSurface2d[0,50])


```
# Load fluxes from files
```

In [ ]:
FluxesLoaded = np.load("fluxes2d.npy")
print(FluxesLoaded.shape)

SurfaceFluxesLoaded = np.load("fluxesSurface2d.npy")
print(FluxesLoaded.shape)


```
# Graph for Flux in particular time columns
```

In [ ]:
%matplotlib widget
fig, ax=plt.subplots()
#np.load("Flux.npy")

ax.plot(wave[indices],(FluxesLoaded[0,:]),'k--',label='0 sec')
ax.plot(wave[indices],(FluxesLoaded[1,:]),'r-',label='0.2 sec')
ax.plot(wave[indices],(FluxesLoaded[2,:]),'b-',label='1.2 sec')
ax.plot(wave[indices],(FluxesLoaded[3,:]),'g-',label='2.2 sec')
ax.plot(wave[indices],(FluxesLoaded[4,:]),'k-',label='4.0 sec')


ax.legend()
plt.xlabel("Wavelenght nm")
plt.ylabel("Flux, W/(m2 Hz)")
#plt.axis([0, 4000, 0, 1.5e-6])

plt.savefig("flux_.svg", format="svg")


In [ ]:
# print(data.ray.intensity[0,0,0])
# print(data.ray.intensity[0,0,1219])
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
#data.ray.wavelength
waveAngs = data.ray.wavelength*10
#waveAngs
indicesAngs = np.arange(len(waveAngs))[(waveAngs > 200) & (waveAngs < 40000)]
print(indicesAngs)


In [ ]:
%matplotlib widget
fig, ax=plt.subplots()
#np.load("Flux.npy")


ax.plot(waveAngs[indicesAngs],(SurfaceFluxesLoaded[0,:]),'k--',label='0 sec')
ax.plot(waveAngs[indicesAngs],(SurfaceFluxesLoaded[1,:]),'r-',label='0.2 sec')
ax.plot(waveAngs[indicesAngs],(SurfaceFluxesLoaded[2,:]),'b-.',label='1.2 sec')
ax.plot(waveAngs[indicesAngs],(SurfaceFluxesLoaded[3,:]),'g-',label='2.2 sec')
ax.plot(waveAngs[indicesAngs],(SurfaceFluxesLoaded[4,:]),'k-',label='4.0 sec')


ax.legend()
plt.xlabel("Wavelenght, Å")
plt.ylabel("Flux, erg/cm^2/s/Å")
#plt.axis([2000, 10000, 0, 1e8])
plt.axis([0, 2000, 0, 3.5e8])

plt.savefig("flux_.svg", format="svg")


In [ ]:
import scipy.integrate as scp
#print(data.ray.intensity.wavelength[1219].data)
#for i in range()
#indicesAngs = indices*10
# wave1[angstrem] = wave[indices*10]
#print(indices[2],indicesAngs[2])
fluxesSum = scp.trapz(SurfaceFluxesLoaded[0,:], x=waveAngs[indicesAngs], dx=1.0, axis=-1)
print(fluxesSum)

In [ ]:
sigma = 5.6784e-8
Teff = np.power(fluxesSum*1e-3/sigma, 1/4)
print(Teff)

# Writing ray.input file
```
# Selecting wavelengths for detailed output
# Choosing all wavelenghts range
# Ray input includes all indexes
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
#print(data.ray.wavelength)
indices = np.arange(len(data.ray.wavelength))
#print(indices)

f = open(rhRepoPath / "rh15d/run/ray.input", "w")  # this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 1)
ololo = 0
for ind in indices:
    # if ololo < 5:
    #     print(ind)
    # ololo += 1
    output += f" {ind}"
#output += ' %i\n' % index500 
f.write(output)
f.close()

#wave = data.ray.wavelength
# indicesAll = np.arange(len(data.ray.wavelength))
# indicesInRange = indicesAll[(wave > 20.0) & (wave < 4000.0)]

#indicesInRange = indicesAll[(wave > 20.0) & (wave < 4000.0)]


# ws1 = wave.sel(wavelength=669, method="nearest")
# ws2 = wave.sel(wavelength=500, method="nearest")
# ws3 = wave.sel(wavelength=100, method="nearest")
# ws4 = wave.sel(wavelength=0, method="nearest")
#print(ws1, ws2, ws3, ws4)

# for i in indices:
#     index f" {i}" = += f" {ind}"
#     output += f" {index669}\n"
# index669 = np.argmin(np.abs(wave.data - 669))
# #If we want to see also one particular wavelength (500 in example)
# index355 = np.argmin(np.abs(wave.data - 355))
# index430 = np.argmin(np.abs(wave.data - 430))

# print(data.ray.wavelength)
# print("-")
# print(data.ray.wavelength_selected)
# print(
#     np.argmin(np.abs(data.ray.wavelength.data - 690))
# )

#print("---")

# print(
#     np.argmin(np.abs(wave.data - 390)),
#     np.argmin(np.abs(wave.data - 395)),
#     np.argmin(np.abs(wave.data - 400))
# )
# x = wave.data - 400
# ii = 0
# for i in x:
#     if i < 50:
#         print(f"{ii}: {np.abs(i)}")
#     ii += 1

#indexesOfWavelengthsRange = {}
#indexesOfWavelengthsRange = []
# for i in range(20,4000,5):
#     #indexesOfWavelengthsRange[i] = np.argmin(np.abs(data.ray.wavelength.data - i))
#     indexesOfWavelengthsRange.append(
#         np.argmin(np.abs(data.ray.wavelength.data - i))
#     )
# for d in indexesOfWavelengthsRange:
#     print(f"{d}: {indexesOfWavelengthsRange[d]}")
#print(len(indexesOfWavelengthsRange))

# for i in range(len(data.ray.wavelength)):
#     indexesOfWavelengthsRange.append(
#         np.argmin(np.abs(data.ray.wavelength.data - i))
#     )

# with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
#     f.write("1.00\n")
#     output = str(len(indexesOfWavelengthsRange))# + 1)
#     for i in indexesOfWavelengthsRange:
#         output += f" {i}"
#     f.write(output)


```
#Need to run RH15D again with a new ray.input file 

# Calculating Flux from Intensity(mu)

```
# reading new output
# writing all tau (time) in one array
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

# print(data.ray.wavelength_indices)
# print(data.ray.source_function.shape)
# print(data.ray.chi.shape)
#print(data.ray.wavelength - data.ray.wavelength[0])
# print(data.ray.wavelength[indices])

indices = np.arange(len(data.ray.wavelength))
print(len(indices))
print(data.atmos.height_scale.shape)

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
indices = np.arange(len(data.ray.wavelength))


# first column
height = data.atmos.height_scale[12, 0].dropna('height')  
 
index = []
for i in indices:
    idx = np.argmin(np.abs(data.ray.wavelength.data - data.ray.wavelength[i].data))
    index.append(idx)
# тут будет 796 одномерных массивов с тау
massivTau = []
for i in indices:
    x = cumtrapz(data.ray.chi[12, 0, :, i].dropna('height'), x=-height)
    x = np.concatenate([[1e-20], x])
    massivTau.append(x)

In [ ]:
print(index[0])
print(len(index))
print("---")
print(massivTau[0])
print(len(massivTau))


In [ ]:
print(data.ray.source_function.shape)
# print(data.ray.source_function[0,0,0,1])
print(len(massivTau))
# print(data.ray.source_function[0,0,0,1219])
x = massivTau[0]
print(x[:])


```
# calculating I(\mu) for 5 \mu
```

In [ ]:
from scipy import integrate

sourceFunction = data.ray.source_function

# mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
# weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
mu = 1.0
intensityMu =[]
# x =
# y =

# --- интрегал
# 796 длин волн
for y in indices:
#     # 249 штук тау
   
    tmpMu = 0
    #for i in range(len(massivTau[y])):
    x = massivTau[y]
    tmpMu = integrate.trapz(sourceFunction[12,0,:,y].data * np.exp(-x[:]/mu), x)

    # tmpMu = integrate.trapz(sourceFunction[40,0,:,y].data * np.exp(-x[:]/mu[1]), x)
        # tmpMu += sourceFunction[0,0,i249,i1644].data * np.exp(-massivTau[i1644][i249]/mu[0]) * (massivTau[i1644+1]-massivTau[i1644])
    # intensityMu.append(1/mu[1] * tmpMu)
    intensityMu.append(1/mu * tmpMu)



print(f"Всего элементоу: {len(intensityMu)}")
print(f"Перший элемент: {intensityMu[0]}")
print(f"Второй элемент: {intensityMu[1]}")
print(f"Последний элемент: {intensityMu[1643]}")

# ---

# fig, ax = plt.subplots()

# ax.plot(data.atmos.height_scale[2,0]/1e6,sourceFunction[2,0,:,0],label='0.2 sec')
#ax.plot(np.log10(massivTau[792]),sourceFunction[22,0,:,0],label='0.2 sec')


In [ ]:
np.save("intensityMu12.npy", intensityMu)

In [ ]:
%matplotlib widget

wave = data.ray.wavelength
ImuLoaded = np.load("IntensityMu12.npy")

# ImuLoaded1 = np.load("IntensityMu-1.npy")
# ImuLoaded2 = np.load("IntensityMu-2.npy")
# ImuLoaded3 = np.load("IntensityMu-3.npy")
# ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
# # print(ImuLoaded1.shape)
fig, ax=plt.subplots()
# #np.load("Flux.npy")



ax.plot(wave[indices], ImuLoaded,'k--',label=r'$\mu$1')

ax.plot(wave[indices], ImuLoaded5,'r-',label=r'$\mu$0.95')
ax.plot(wave[indices],data.ray.intensity[2,0,indices],'g-.',label='12 sec')

# ax.plot(wave[indices], ImuLoaded1,'k--',label=r'$\mu$1')
# ax.plot(wave[indices], ImuLoaded2,'k-',label=r'$\mu$2')
# ax.plot(wave[indices], ImuLoaded3,'g-',label=r'$\mu$3')
# ax.plot(wave[indices], ImuLoaded4,'r-',label=r'$\mu$4')
# ax.plot(wave[indices], ImuLoaded5,'b-',label=r'$\mu$5')

ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
fluxesImu =[]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]

ImuLoaded1 = np.load("IntensityMu-1.npy")
ImuLoaded2 = np.load("IntensityMu-2.npy")
ImuLoaded3 = np.load("IntensityMu-3.npy")
ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
Imuloaded = []
Imuloaded.append(ImuLoaded1)
Imuloaded.append(ImuLoaded2)
Imuloaded.append(ImuLoaded3)
Imuloaded.append(ImuLoaded4)
Imuloaded.append(ImuLoaded5)
#print(len(Imuloaded))
for i in indices:
    imu = 0
    for i5 in range(len(weightMu)):
        imu += Imuloaded[i5][i]*weightMu[i5]
    fluxesImu.append(imu)
print(len(fluxesImu))


In [ ]:
np.save("fluxesImu40.npy", fluxesImu)

In [ ]:
%matplotlib widget

fluxesImu1 = np.load("fluxesImu0.npy")
fluxesImu2 = np.load("fluxesImu2.npy")
fluxesImu3 = np.load("fluxesImu12.npy")
fluxesImu4 = np.load("fluxesImu22.npy")
fluxesImu5 = np.load("fluxesImu40.npy")



fig, ax=plt.subplots()
#np.load("Flux.npy")

wave = data.ray.wavelength

ax.plot(wave[indices], fluxesImu1,'k--',label='0 sec')
ax.plot(wave[indices], fluxesImu2,'k-',label='0.2 sec')
ax.plot(wave[indices], fluxesImu3,'r-',label='1.2 sec')
ax.plot(wave[indices], fluxesImu4,'b-',label='2.2 sec')
ax.plot(wave[indices], fluxesImu5,'g-',label='4.0 sec')


ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Flux ($W/(m^2*Hz)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("FluxperHz_.svg", format="svg")
# ax.plot(wave[indices], ImuLoaded5)
# plt.xlabel("Wavelenght, nm")
# plt.ylabel("Flux, W/cm^2")
#plt.xlim(200,1000)
# plt.axis([2000, 10000, 0, 1e8])
#plt.axis([200, 1000,0, 1e-7])


In [ ]:
const1 = 2.99792458e+21
fluxesImu = np.load("fluxesImu40.npy")
fluxesSurfaceMu = []
# fluxesSurfaceMu.append(const1 * fluxesImu[0] /(np.power(wave[0] * 10,2)))
# print(fluxesSurfaceMu)
for i in indices:
    fluxesSurfaceMu.append(const1 * fluxesImu[i] /(np.power(wave[i] * 10,2)))

np.save("fluxesSurfaceMu40.npy", fluxesSurfaceMu)

In [ ]:
from scipy import integrate
fluxesSurfaceMu = np.load("fluxesSurfaceMu0.npy")

#print(data.ray.intensity.wavelength[1219].data)
#for i in range()
# indicesAngs = indices*10
# wave1[angstrem] = wave[indices*10]
#print(indices[2],indicesAngs[2])
x = wave[indices]
fluxesSum = integrate.trapz(fluxesSurfaceMu[:], x * 10)
print(fluxesSum)

In [ ]:
sigma = 5.6784e-8
Teff = np.power(fluxesSum*1e-3/sigma, 1/4)
print(Teff)

In [ ]:
fig, ax=plt.subplots()

ax.plot(wave[indices], fluxesSurfaceMu)
plt.xlabel("Wavelenght (nm)")
plt.ylabel("Flux")
plt.xlim(200,1000)
plt.ylim(0,8e8)


In [ ]:
fluxesSurfaceMu1 = np.load("fluxesSurfaceMu0.npy")
fluxesSurfaceMu2 = np.load("fluxesSurfaceMu2.npy")
fluxesSurfaceMu3 = np.load("fluxesSurfaceMu12.npy")
fluxesSurfaceMu4 = np.load("fluxesSurfaceMu22.npy")
fluxesSurfaceMu5 = np.load("fluxesSurfaceMu40.npy")



fig, ax=plt.subplots()

ax.plot(wave[indices]*10, fluxesSurfaceMu1,'k--',label='0 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu2,'k-',label='0.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu3,'r--',label='1.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu4,'b--',label='2.2 sec')
ax.plot(wave[indices]*10, fluxesSurfaceMu5,'g--',label='4.0 sec')

ax.legend()
plt.xlabel("Wavelenght, Å")
plt.ylabel("Flux, erg/cm^2/s/Å")
#plt.axis([2000, 9000, 0, 6.5e8])
#plt.axis([0, 2000, 0, 1e7])

plt.savefig("flux_.svg", format="svg")

```
#To make a plot for intensity at a fixed wavelength
```

In [ ]:
%matplotlib widget
data.ray.intensity.sel(wavelength=500, method='nearest').plot()

# Tau=1 for wavelenght

In [ ]:
%matplotlib widget

#data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")


fig, ax = plt.subplots()
ax.plot(height / 1e6, massivTau[20])  # height in Mm

plt.ylabel(r'$\tau$$_{20}$')
plt.xlabel('H (Mm)')

ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
#ax.axvline(x=0.5,linestyle='dashed',color='black')
plt.savefig("tau360_.svg", format="svg")


# Plot the departure coefficients for the levels of Ca II



In [ ]:
%matplotlib widget
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

plt.figure()
for i in range(5):
    plt.plot(data.atmos.height_scale[22,0]/1e6, 
             data.atom_CA.populations[i, 22, 0]/data.atom_CA.populations_LTE[i, 22, 0],
             label='Level %i' % (i + 1))
plt.legend(loc="upper left")
plt.ylabel("Departure coefficients at 0 sec")
plt.xlabel("H (Mm)")
#plt.axis([0, 0.4])

# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# fig, ax = plt.subplots()
# ax.plot(tau500,data.atom_CA.populations[1, 0, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0, 0], label='Level %1' % (1)) 
# plt.legend(loc="upper left")
# plt.xlabel("tau500")
# plt.ylim(0.05, 2)
# plt.ylabel("Departure coefficients")
# plt.ylim(0.05, 2)
plt.axis([0, 0.5, 0, 1e5])


In [ ]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, data.atom_CA.populations[0,0,0,:].dropna('height')/data.atom_CA.populations_LTE[0,0,0,:].dropna('height'),'r--',label=r'b$_{l}$')
ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, np.log10(data.atom_CA.populations[0,3,0,:].dropna('height')/data.atom_CA.populations_LTE[0,3,0,:].dropna('height')),'k-',label=r'b$_{u}$')
plt.legend(loc='best')
plt.xlabel('H (Mm)')
plt.ylabel(r'Log (b$_{i}$)')
#plt.xlim([0,2])

# tau=1

In [ ]:
%matplotlib widget

#data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")


fig, ax = plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6,'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6,'k-',label='0.2 sec')
#ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6,'r-',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6,'b-',label='2.2 sec')
#ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6,'g-',label='4 sec')

plt.legend(loc='best')

plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')


In [ ]:
height = data.ray.height
fig, ax = plt.subplots()
ax.plot(height/)


# от тау 500

In [ ]:
# xVals = data.atmos.height_scale[0,0]/1e6
# with open("/tmp/x.txt", "w") as f:
#     for x in xVals.data:
#         f.write(f"{x}\n")

print(data.atom_CA.populations[0,22,0])
# yVals = data.atom_CA.populations[0,0,0]
# with open("/tmp/y.txt", "w") as f:
#     for y in yVals.data:
#         f.write(f"{y}\n")


#print(data.atom_CA.populations[0,0,0][247])
# print("- - - - -")
# print(data.atom_CA.populations[1,0,0])
# print("- - - - -")
# print(data.atom_CA.populations[2,0,0])

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(
    data.atmos.height_scale[0,0]/1e6,
    data.atom_CA.populations[0,0,0]
)

def animate(i):
    #line.set_xdata(x)
    line.set_ydata(data.atom_CA.populations[0, i, 0][0])
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    np.arange(52),
    #blit=True,
    repeat=False
)
plt.show()

In [ ]:
%matplotlib widget

plt.figure()
for i in range(5):
    plt.plot(
        data.atmos.height_scale[22,0]/1e6, 
        data.atom_CA.populations[i, 22, 0]/data.atom_CA.populations_LTE[i, 22, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.xlim(0,4)
plt.ylim(-20,1e6)
#/data.atom_CA.populations_LTE[i, 100, 0]

# Departure coefficient vs optical depth

In [ ]:
%matplotlib widget

for i in range(6):
    plt.plot(np.log10(tau669), np.log10(data.atom_CA.populations[i,22,0,:].dropna('height')/data.atom_CA.populations_LTE[i,22,0,:].dropna('height')),label=r'b'+r'$_{'+str(i)+'}$')
    plt.legend(loc='best')
    plt.xlabel(r'Log ($\tau$$_{669}$)')
    plt.ylabel(r'Log (b$_{i}$)')
    plt.xlim([-12,2])

# Source function

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

rh15d_vis.SourceFunction(data);
plt.xlim(-0.2,0.4)
plt.ylim(0,1e6)

In [ ]:
%matplotlib widget

data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()

sourceFunction = hf['source_function']

fig, ax = plt.subplots()

# ax.plot(data.atmos.height_scale[2,0]/1e6,sourceFunction[2,0,:,0],label='0.2 sec')
ax.plot(np.log10(tau669),sourceFunction[2,0,:,0],label='0.2 sec')

#data.source_function.data

# Различные блоки, которые не пригодились или не работают

In [ ]:
# from scipy import integrate
# from integrate import quad, dblquad
from scipy.integrate import quad, dblquad
fluxes = []

# def u(x):
#     #print(i, ii)
#     return np.cos(x)*data.ray.intensity[i,0,ii].data
#print(u(0))
#x = np.cos(y)
#print(data.ray.intensity)
#print(data.ray.intensity.shape, data.ray.intensity.count().data)
for i in range(data.ray.intensity.x.count().data):
    for ii in range(data.ray.intensity.wavelength.count().data):
        #print(data.ray.intensity[i,0,ii].data)
        flx, err = dblquad(lambda x, y: np.sin(x)*np.cos(x)*data.ray.intensity[i,0,ii].data, 0, np.pi, lambda x: 0, lambda x: np.pi/2)
        #integrate.quad(lambda x: x*data.ray.intensity[i,0,ii].data, -1, 1) #weight="cos", wvar=1)
        #fluxes.append(round(flx,5))
        #print(flx)
        fluxes.append(flx)
#print(fluxes)
np.save("Flux.npy",fluxes)

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d
import h5py

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig1, ax1=plt.subplots()
#ax1.plot(vz1t[:,22],np.log10(tg1t[:,22]))
ax1.plot(vz1t[:,0],(tg1t[:,0]))

plt.xlabel('Height (Mm)')
plt.ylabel('(Velocity (m/s))')

#plt.ylabel('lg(Temperature (K))')
#plt.title('Figure 1’)

# further nonsence with atmo


In [ ]:
rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5");
#rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5");
# import h5py
#   data = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
#   for group in data.keys():
#        print(group)

In [ ]:
atmosTemp = atmos.temperature 
#atmostemp

# print(atmosTemp[0,:,:,248])
# print()
# print(atmosTemp[0,:,:,247])

# 1 - nt(1) - ?
# 2 - nx(52) - время
# 3 - ny(1) - ?
# 4 - nz(249) - высота

# sm1 = atmosTemp[0,0,0,248]
# print(sm1)
# sm2 = atmosTemp[0,0,0]
# print(sm2)

#print(atmosTemp[0,:,:,:])

# ---

atmosHeight = atmos.z
atmosHeight

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
#fig.subplots_adjust(hspace = 0.1,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
#axs.ravel()

# im=axs[0].imshow(atmosTemp[0,:,:,248],cmap='plasma')
# im=axs[1].imshow(atmosTemp[0,:,:,247],cmap='plasma')
# im=axs[2].imshow(atmosTemp[0,:,:,246],cmap='plasma')

im=axs[0].plot(atmosTemp[0,:,:,247])
im=axs[1].plot(atmosHeight[0,:,:,247])
#im=axs[2].plot(atmosTemp[0,:,:,246])

# divider0 = make_axes_locatable(axs[0])
# cax0 = divider0.append_axes("right", size="5%", pad=0.05)
# cbar0=plt.colorbar(im,cax=cax0)
# cbar0.set_label('T (K)')

# divider1 = make_axes_locatable(axs[1])
# cax1 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar1=plt.colorbar(im,cax=cax1)
# cbar1.set_label('T (K)')

#axs[0].set_xlabel('pixels')
axs[0].set_ylabel('pixels')

In [ ]:
import h5py
data = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
for group in data.keys():
    print(group)

In [ ]:
import xarray
atmos = xarray.open_dataset("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5")
atmos

In [ ]:
atmos.temperature.plot()

In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/uio/project-Mflare/mflares/"

from helita.sim import bifrost
data = bifrost.BifrostData("radyn_out.F13_dpl_GRID_pt1s", snap=300, fdir=dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")

In [ ]:
#H = hf['z'][0,:] 
# geometric height
plt.figure()
plt.plot(
        atmos.height_scale[0,0]/1e6, 
        atmos.electron_density[0, 0, 0]
)
print(atmos.height_scale[0,0]/1e6)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# height = hf['z'][0,:,0,:] # geometric height
# time = hf['x'][:]
# temp = hf['temperature']
# vlos = hf['velocity_z'] # LOS_V
# eden = hf['electron_density']

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, axs =plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.25,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
axs.ravel()
im=axs[0].matshow(tg1t)
divider1 = make_axes_locatable(axs[0])
cax0 = divider1.append_axes("right", size="5%", pad=0.05)
cbar=plt.colorbar(im,cax=cax0)
cbar.set_label('T (K)')
axs[0].set_xlabel('time')
axs[0].set_ylabel('height')

# im=axs[1].matshow(vlos[0,:,0,:]/1e3,cmap='bwr',vmax=1.5,vmin=-1.5)
# divider1 = make_axes_locatable(axs[1])
# cax0 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar=plt.colorbar(im,cax=cax0)
# cbar.set_label('V (km/s)')
# axs[1].set_xlabel('height')
# axs[1].set_ylabel('time')

# im=axs[2].matshow(eden[0,:,0,:]*1e4,cmap='BrBG',vmax=20,vmin=-100)
# divider1 = make_axes_locatable(axs[2])
# cax0 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar=plt.colorbar(im,cax=cax0)
# cbar.set_label('electron density (G)')
# axs[2].set_xlabel('height')
# axs[2].set_ylabel('time')